In [1]:
# ============================================================
# 🚀 GOOGLE COLAB SETUP - Run this cell first!
# ============================================================
# If running on Google Colab, uncomment and run the lines below.
# Binder users: packages are automatically installed from requirements.txt

# !pip install -q -r https://raw.githubusercontent.com/arunissun/Montandon-Data-Fetching-Examples/master/requirements.txt

# # Set your API token (you'll be prompted to enter it securely)
# import os
# from getpass import getpass
# if 'MONTANDON_API_TOKEN' not in os.environ:
#     os.environ['MONTANDON_API_TOKEN'] = getpass('Enter your Montandon API token: ')


## 1. Setup and Installation

First, let's import the required libraries.

In [2]:
# Import libraries
import pandas as pd
from pystac_client import Client
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Any
import warnings
import os
from getpass import getpass
import json
warnings.filterwarnings('ignore')



In [3]:
# Set display options for better readability
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.width', None)

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("All libraries imported successfully!")

All libraries imported successfully!


---

## 2. Connecting to the Montandon STAC API

Let's establish a connection to the API and explore its basic structure.

In [4]:
# Connect to Montandon STAC API with Authentication
STAC_API_URL = "https://montandon-eoapi-stage.ifrc.org/stac"

# Get authentication token
# Option 1: From environment variable (recommended for automation)
api_token = os.getenv('MONTANDON_API_TOKEN')

# Option 2: Prompt user for token if not in environment
if api_token is None:
    print("=" * 70)
    print("AUTHENTICATION REQUIRED")
    print("=" * 70)
    print("\nThe Montandon STAC API requires a Bearer Token for authentication.")
    print("\nHow to get your token:")
    print("  1. Visit: https://goadmin-stage.ifrc.org/")
    print("  2. Log in with your IFRC credentials")
    print("  3. Generate an API token from your account settings")
    print("\nAlternatively, set the MONTANDON_API_TOKEN environment variable:")
    print("  PowerShell: $env:MONTANDON_API_TOKEN = 'your_token_here'")
    print("  Bash: export MONTANDON_API_TOKEN='your_token_here'")
    print("\n" + "=" * 70)
    
    # Prompt for token (hidden input)
    api_token = getpass("Enter your Montandon API Token: ")
    
    if not api_token or api_token.strip() == "":
        raise ValueError("API token is required to access the Montandon STAC API")

# Create authentication headers
auth_headers = {"Authorization": f"Bearer {api_token}"}

# Connect to STAC API with authentication
try:
    client = Client.open(STAC_API_URL, headers=auth_headers)
    print(f"\n[OK] Connected to: {STAC_API_URL}")
    print(f"[OK] API Title: {client.title}")
    print(f"[OK] Authentication: Bearer Token (OpenID Connect)")
    print(f"[OK] Auth Provider: https://goadmin-stage.ifrc.org/o/.well-known/openid-configuration")
except Exception as e:
    print(f"\n[ERROR] Authentication failed: {e}")
    print("\nPlease check:")
    print("  1. Your token is valid and not expired")
    print("  2. You have the correct permissions")
    print("  3. The API endpoint is accessible")
    raise

AUTHENTICATION REQUIRED

The Montandon STAC API requires a Bearer Token for authentication.

How to get your token:
  1. Visit: https://goadmin-stage.ifrc.org/
  2. Log in with your IFRC credentials
  3. Generate an API token from your account settings

Alternatively, set the MONTANDON_API_TOKEN environment variable:
  PowerShell: $env:MONTANDON_API_TOKEN = 'your_token_here'
  Bash: export MONTANDON_API_TOKEN='your_token_here'


[OK] Connected to: https://montandon-eoapi-stage.ifrc.org/stac
[OK] API Title: stac-fastapi
[OK] Authentication: Bearer Token (OpenID Connect)
[OK] Auth Provider: https://goadmin-stage.ifrc.org/o/.well-known/openid-configuration


### Understanding API Conformance

The API conforms to various STAC specifications. Let's check what capabilities are available:

In [5]:
# Check API conformance and capabilities
# Note: The API supports STAC specifications - conformance details available via API endpoints
print("API Connection Verified")
print("=" * 60)
print(f"Connected to: {STAC_API_URL}")
print(f"API Title: {client.title}")
print("\nThe API is ready for data queries.")
print("You can now explore collections and search for disaster event data.")

API Connection Verified
Connected to: https://montandon-eoapi-stage.ifrc.org/stac
API Title: stac-fastapi

The API is ready for data queries.
You can now explore collections and search for disaster event data.


In [6]:
# Define helper function using pystac_client's client.search() method
from typing import Optional

def search_stac(
    collections: Optional[List[str]] = None,
    max_items: int = 100,
    bbox: Optional[List[float]] = None,
    datetime_range: Optional[str] = None,
    query: Optional[Dict[str, Any]] = None,
    sortby: Optional[List[str]] = None,
    filter_body: Optional[Dict[str, Any]] = None,
    filter_lang: Optional[str] = None,
) -> list:
    """
    Search STAC API using pystac_client's client.search() method.
    
    This uses the standard pystac_client search interface which handles
    pagination automatically.
    
    Parameters:
    -----------
    collections : list of str
        Collection IDs to search
    max_items : int
        Maximum number of results to return
    bbox : list of float
        Bounding box [min_lon, min_lat, max_lon, max_lat]
    datetime_range : str
        ISO 8601 datetime range string (e.g., '2024-01-01/2024-12-31')
    query : dict
        Query parameters for filtering
    sortby : list of str
        Sort order (e.g., [{'field': 'properties.datetime', 'direction': 'desc'}])
    filter_body : dict
        CQL2 filter expression
    filter_lang : str
        Filter language (e.g., 'cql2-json')
    
    Returns:
    --------
    list : List of pystac Item objects matching the search criteria
    """
    # Build search parameters
    search_params = {
        "max_items": max_items
    }
    
    if collections:
        search_params["collections"] = collections
    if bbox:
        search_params["bbox"] = bbox
    if datetime_range:
        search_params["datetime"] = datetime_range
    if query:
        search_params["query"] = query
    if sortby:
        search_params["sortby"] = sortby
    if filter_body:
        search_params["filter"] = filter_body
    if filter_lang:
        search_params["filter_lang"] = filter_lang
    
    # Use client.search() - pagination is handled automatically
    search = client.search(**search_params)
    
    # Return list of items
    return list(search.items())

print("✅ PySTAC helper function initialized (using client.search())")

✅ PySTAC helper function initialized (using client.search())


---

## 3. Exploring Available Collections

Collections in the Montandon API are organized by data source and type:

- **Events**: Main disaster event records
- **Hazards**: Specific hazard information (earthquakes, floods, etc.)
- **Impacts**: Impact assessments (casualties, economic losses, etc.)

Let's explore all available collections:

In [7]:
# Get all collections using pystac_client
# The client handles authentication and pagination automatically

print("Fetching all collections...")

# Get all collections using the client's get_collections() method
collections_list = list(client.get_collections())

print(f"\n{'='*60}")
print(f"Total Collections Available: {len(collections_list)}")
print(f"{'='*60}\n")

# Create a DataFrame for better visualization
collection_data = []
for coll in collections_list:
    desc = coll.description or 'N/A'
    if desc and desc != 'N/A' and len(desc) > 80:
        desc = desc[:80] + '...'
    
    collection_data.append({
        'Collection ID': coll.id,
        'Title': coll.title or 'N/A',
        'Description': desc,
        'License': coll.license or 'N/A'
    })

collections_df = pd.DataFrame(collection_data)
collections_df

Fetching all collections...

Total Collections Available: 29



,Collection ID,Title,Description,License
0,desinventar-events,DesInventar Mapped Events,Events mapped from the DesInventar disaster lo...,proprietary
1,desinventar-impacts,DesInventar Impacts,Impact records mapped from the DesInventar dis...,proprietary
2,emdat-events,EM-DAT Source Events,Global Disaster Events from the Emergency Even...,MIT
3,emdat-hazards,EM-DAT Source Hazards,Hazard records from the Emergency Events Datab...,MIT
4,emdat-impacts,EM-DAT Source Impacts,Impact records from the Emergency Events Datab...,MIT
...,...,...,...,...
24,pdc-impacts,PDC Impacts,Impact records from the Pacific Disaster Cente...,TBD
25,reference-events,Reference Events,A collection of reference events loaded into M...,Apache-2.0
26,usgs-events,USGS Events,Events from the United States Geological Surve...,Apache-2.0
27,usgs-hazards,USGS Hazards,Hazard records from the United States Geologic...,Apache-2.0


### Categorizing Collections by Type

Let's organize collections by their type (events, hazards, impacts):

In [8]:
# Categorize collections
events_collections = [c for c in collections_list if '-events' in c.id]
hazards_collections = [c for c in collections_list if '-hazards' in c.id]
impacts_collections = [c for c in collections_list if '-impacts' in c.id]

print("Collections by Type:\n")
print(f"Events Collections ({len(events_collections)}):")
for col in events_collections:
    print(f"   • {col.id}")

print(f"Hazards Collections ({len(hazards_collections)}):")
for col in hazards_collections:
    print(f"   • {col.id}")

print(f"Impacts Collections ({len(impacts_collections)}):")
for col in impacts_collections:
    print(f"   • {col.id}")

Collections by Type:

Events Collections (12):
   • desinventar-events
   • emdat-events
   • gdacs-events
   • gfd-events
   • glide-events
   • ibtracs-events
   • idmc-gidd-events
   • idmc-idu-events
   • ifrcevent-events
   • pdc-events
   • reference-events
   • usgs-events
Hazards Collections (8):
   • emdat-hazards
   • gdacs-hazards
   • gfd-hazards
   • glide-hazards
   • ibtracs-hazards
   • ifrcevent-hazards
   • pdc-hazards
   • usgs-hazards
Impacts Collections (9):
   • desinventar-impacts
   • emdat-impacts
   • gdacs-impacts
   • gfd-impacts
   • idmc-gidd-impacts
   • idmc-idu-impacts
   • ifrcevent-impacts
   • pdc-impacts
   • usgs-impacts


### Categorizing by Data Source

In [9]:
# Extract data sources
sources = {}
for col in collections_list:
    source = col.id.split('-')[0]
    if source not in sources:
        sources[source] = []
    sources[source].append(col.id)

print("Collections by Data Source:\n")
for source, cols in sorted(sources.items()):
    print(f"{source.upper()} ({len(cols)} collections):")
    for col in sorted(cols):
        print(f"   • {col}")
    print()

Collections by Data Source:

DESINVENTAR (2 collections):
   • desinventar-events
   • desinventar-impacts

EMDAT (3 collections):
   • emdat-events
   • emdat-hazards
   • emdat-impacts

GDACS (3 collections):
   • gdacs-events
   • gdacs-hazards
   • gdacs-impacts

GFD (3 collections):
   • gfd-events
   • gfd-hazards
   • gfd-impacts

GLIDE (2 collections):
   • glide-events
   • glide-hazards

IBTRACS (2 collections):
   • ibtracs-events
   • ibtracs-hazards

IDMC (4 collections):
   • idmc-gidd-events
   • idmc-gidd-impacts
   • idmc-idu-events
   • idmc-idu-impacts

IFRCEVENT (3 collections):
   • ifrcevent-events
   • ifrcevent-hazards
   • ifrcevent-impacts

PDC (3 collections):
   • pdc-events
   • pdc-hazards
   • pdc-impacts

REFERENCE (1 collections):
   • reference-events

USGS (3 collections):
   • usgs-events
   • usgs-hazards
   • usgs-impacts



### Deep Dive: Examining a Specific Collection

Let's examine the GDACS hazards collection in detail:

In [10]:
# Get a specific collection using client.get_collection()
collection_id = 'gdacs-hazards'

# Fetch collection details using pystac_client
gdacs_hazards = client.get_collection(collection_id)

print(f"Collection Details: {collection_id}\n")
print(f"Title: {gdacs_hazards.title or 'N/A'}")
print(f"Description: {gdacs_hazards.description or 'N/A'}")
print(f"License: {gdacs_hazards.license or 'N/A'}")

# Spatial extent
if gdacs_hazards.extent:
    if gdacs_hazards.extent.spatial:
        print(f"\nSpatial Extent:")
        print(f"   Bounding Box: {gdacs_hazards.extent.spatial.bboxes}")
    
    if gdacs_hazards.extent.temporal:
        print(f"\nTemporal Extent:")
        print(f"   Intervals: {gdacs_hazards.extent.temporal.intervals}")

# Additional properties (keywords, providers, etc.)
if gdacs_hazards.keywords:
    print(f"\nKeywords: {', '.join(gdacs_hazards.keywords)}")

if gdacs_hazards.providers:
    print(f"\nProviders:")
    for provider in gdacs_hazards.providers:
        print(f"   • {provider.name} ({provider.roles})")

Collection Details: gdacs-hazards

Title: GDACS Hazards
Description: Hazard records from the Global Disaster Alert and Coordination System (GDACS). GDACS is a cooperation framework between the United Nations, the European Commission and disaster managers worldwide to improve alerts, information exchange and coordination in the first phase after major sudden-onset disasters. It provides detailed hazard information including affected areas, alert levels, and severity scores for earthquakes, tsunamis, floods, tropical cyclones, volcanic eruptions, and wildfires. Each hazard includes specific information based on the hazard type and uses specialized models for assessment. More information on the GDACS mapping in Monty can be found in the [GDACS Hazard Source Mappings](https://github.com/IFRCGo/monty-stac-extension/tree/main/model/sources/GDACS#hazard-item).
License: MIT

Spatial Extent:
   Bounding Box: [[-180, -90, 180, 90]]

Temporal Extent:
   Intervals: [[datetime.datetime(2000, 1, 1, 

---

## 4. Basic Data Retrieval - Simple Queries

Now let's start retrieving actual data! We'll begin with simple queries.

### Example 1: Retrieve 10 Items from GDACS Hazards

In [11]:
# Simple search using client.search()
try:
    search = client.search(
        collections=["gdacs-hazards"],
        max_items=10
    )
    items = list(search.items())
    
    print(f"✅ Retrieved {len(items)} items from gdacs-hazards collection\n")
    
    # Display first item details
    if items:
        first_item = items[0]
        print("📄 First Item Details:")
        print(f"ID: {first_item.id}")
        print(f"Collection: {first_item.collection_id}")
        print(f"Geometry Type: {first_item.geometry.get('type', 'None')}")
        print(f"\nProperties:")
        for key, value in list(first_item.properties.items())[:10]:  # Show first 10 properties
            print(f"  • {key}: {value}")
    else:
        print("No items found in the search results")
        
except Exception as e:
    print(f"❌ Search failed: {e}")

✅ Retrieved 10 items from gdacs-hazards collection

📄 First Item Details:
ID: gdacs-hazard-1514353-1676468
Collection: gdacs-hazards
Geometry Type: Point

Properties:
  • roles: ['source', 'hazard']
  • title: Earthquake in Colombia
  • source: NEIC
  • datetime: 2025-12-10T08:27:06Z
  • keywords: ['Geological', 'Seismic', 'COL', 'Earthquake']
  • description: Green M 5.5 Earthquake in Colombia at: 10 Dec 2025 08:27:06.
  • end_datetime: 2025-12-10T08:27:06Z
  • monty:etl_id: 6c6185c9-68a6-4f70-863c-5bcfed162c25
  • severitydata: {'severity': 5.5, 'severitytext': 'Magnitude 5.5M, Depth:140.692km', 'severityunit': 'M'}
  • monty:corr_id: 20251210-COL-GH0101-1676468-GCDB


### Example 2: Using Direct API Requests

Sometimes you might want to use direct HTTP requests instead of the Python client:

In [12]:
# Direct search using pystac_client's client.search() method

# Perform search using client.search()
search = client.search(
    collections=["gdacs-hazards"],
    max_items=5
)
items_list = list(search.items())

print(f"📡 PySTAC Client Response")
print(f"\n📊 Results:")
print(f"  • Number of Items: {len(items_list)}")
print(f"  • Collection: gdacs-hazards")

# Display feature IDs
if items_list:
    print(f"\n Item IDs:")
    for item in items_list:
        print(f"  • {item.id}")

📡 PySTAC Client Response

📊 Results:
  • Number of Items: 5
  • Collection: gdacs-hazards

 Item IDs:
  • gdacs-hazard-1514353-1676468
  • gdacs-hazard-1514350-1676464
  • gdacs-hazard-1514341-1676453
  • gdacs-hazard-1514332-1676442
  • gdacs-hazard-1514294-1676432


---

## 5. Advanced Filtering Techniques

Now let's explore more sophisticated filtering options.

### 5.1 Temporal Filtering with DateTime

Filter events by date range:

In [13]:
# Define date range - last 90 days
end_date = datetime.now()
start_date = end_date - timedelta(days=90)

# Format as ISO 8601
datetime_range = f"{start_date.strftime('%Y-%m-%dT%H:%M:%SZ')}/{end_date.strftime('%Y-%m-%dT%H:%M:%SZ')}"

print(f"📅 Searching for events between:")
print(f"   Start: {start_date.strftime('%Y-%m-%d')}")
print(f"   End: {end_date.strftime('%Y-%m-%d')}\n")

# Search with datetime filter using client.search()
search = client.search(
    collections=['gdacs-events'],
    datetime=datetime_range,
    max_items=20
)
items = list(search.items())

print(f"✅ Found {len(items)} events in the last 90 days")

# Display event details
if items:
    print("\n📋 Recent Events:")
    for i, item in enumerate(items[:5], 1):
        event_date = item.properties.get('datetime', 'N/A')
        event_name = item.properties.get('title', item.id)
        print(f"  {i}. {event_name} - {event_date}")

📅 Searching for events between:
   Start: 2025-09-11
   End: 2025-12-10

✅ Found 20 events in the last 90 days

📋 Recent Events:
  1. Earthquake in Colombia - 2025-12-10T08:27:06Z
  2. Earthquake in Indonesia - 2025-12-10T07:42:55Z
  3. Earthquake in Chile - 2025-12-10T06:38:27Z
  4. Earthquake in Volcano Islands, Japan Region - 2025-12-10T05:18:49Z
  5. Earthquake in Russia - 2025-12-09T23:19:52Z


### 5.2 Spatial Filtering with Bounding Box (bbox)

Filter events by geographic area. Let's search for events in Southeast Asia:

In [14]:
# Define bounding box for Southeast Asia [min_lon, min_lat, max_lon, max_lat]
southeast_asia_bbox = [95.0, -10.0, 141.0, 28.0]

print("🗺️ Searching for events in Southeast Asia")
print(f"   Bounding Box: {southeast_asia_bbox}\n")

# Search with bbox using client.search()
search = client.search(
    collections=['gdacs-events', 'emdat-events', 'usgs-events'],
    bbox=southeast_asia_bbox,
    max_items=30
)
items = list(search.items())

print(f"✅ Found {len(items)} events in Southeast Asia")

# Analyze by collection
if items:
    collection_counts = {}
    for item in items:
        col = item.collection_id
        collection_counts[col] = collection_counts.get(col, 0) + 1
    
    print("\n📊 Events by Collection:")
    for col, count in sorted(collection_counts.items()):
        print(f"  • {col}: {count} events")

🗺️ Searching for events in Southeast Asia
   Bounding Box: [95.0, -10.0, 141.0, 28.0]

✅ Found 30 events in Southeast Asia

📊 Events by Collection:
  • gdacs-events: 14 events
  • usgs-events: 16 events


### 5.3 Combining Multiple Filters

Let's combine temporal and spatial filters:

In [15]:
# Search for recent events in Europe
europe_bbox = [-10.0, 35.0, 40.0, 71.0]
recent_datetime = "2024-01-01T00:00:00Z/2024-12-31T23:59:59Z"

print("🔍 Searching for 2024 events in Europe\n")

search = client.search(
    collections=['gdacs-events', 'emdat-events'],
    bbox=europe_bbox,
    datetime=recent_datetime,
    max_items=50
)
items = list(search.items())

print(f"✅ Found {len(items)} events in Europe during 2024")

# Display sample
if items:
    print("\n📋 Sample Events:")
    for i, item in enumerate(items[:3], 1):
        print(f"\n  {i}. ID: {item.id}")
        print(f"     Collection: {item.collection_id}")
        print(f"     Title: {item.properties.get('title', 'N/A')}")
        print(f"     Date: {item.properties.get('datetime', 'N/A')}")
        print(f"     Country: {item.properties.get('monty:country_codes', 'N/A')}")

🔍 Searching for 2024 events in Europe

✅ Found 50 events in Europe during 2024

📋 Sample Events:

  1. ID: emdat-event-2024-0960-TUN
     Collection: emdat-events
     Title: Water in Tunisia of December 2024
     Date: 2024-12-31T00:00:00Z
     Country: ['TUN']

  2. ID: emdat-event-2024-0954-ITA
     Collection: emdat-events
     Title: Water in Italy of December 2024
     Date: 2024-12-31T00:00:00Z
     Country: ['ITA']

  3. ID: emdat-event-2024-0949-ESP
     Collection: emdat-events
     Title: Water in Spain of December 2024
     Date: 2024-12-26T00:00:00Z
     Country: ['ESP']


### 5.4 Advanced Filtering with CQL2

Common Query Language (CQL2) allows for complex filtering logic. Let's search for flood events in Spain (like the example from the documentation):

### 5.5 Finding Hazards Associated with an Event

Once we have an event, we can find all related hazards using the correlation ID:

In [16]:
# CQL2 filter for Spain flood events
spain_flood_late_oct_filter = {
    "op": "and",
    "args": [
        {
            "op": "a_contains",
            "args": [
                {"property": "monty:country_codes"},
                "ESP"
            ]
        },
        {
            "op": "a_overlaps",
            "args": [
                {"property": "monty:hazard_codes"},
                ["nat-hyd-flo-flo", "FL"]
            ]
        },
        {
            "op": "t_intersects",
            "args": [
                {"property": "datetime"},
                {"interval": ["2024-10-20T00:00:00Z", "2024-11-05T23:59:59Z"]}
            ]
        }
    ]
}

# Search using client.search() with CQL2 filter
search = client.search(
    collections=["gdacs-events", "emdat-events", "glide-events", "reference-events"],
    filter=spain_flood_late_oct_filter,
    filter_lang="cql2-json",
    max_items=100
)
spain_flood_late_oct_events = list(search.items())

print(f"✅ Found {len(spain_flood_late_oct_events)} flood events in Spain (Oct 20 - Nov 5, 2024)\n")

if spain_flood_late_oct_events:
    print(f"📋 All {len(spain_flood_late_oct_events)} Spain Late October/Early November Flood Events:\n")
    for i, event in enumerate(spain_flood_late_oct_events, 1):
        print(f"\n{i}. Event Details:")
        print(f"   ID: {event.id}")
        print(f"   Title: {event.properties.get('title', 'N/A')}")
        print(f"   Date: {event.properties.get('datetime', 'N/A')}")
        print(f"   Collection: {event.collection_id}")
        print(f"   Countries: {event.properties.get('monty:country_codes', 'N/A')}")
        print(f"   Hazard Codes: {event.properties.get('monty:hazard_codes', 'N/A')}")
        print(f"   Correlation ID: {event.properties.get('monty:corr_id', 'N/A')}")

else:    print("⚠️ No flood events found in Spain during Oct 20 - Nov 5, 2024")

✅ Found 4 flood events in Spain (Oct 20 - Nov 5, 2024)

📋 All 4 Spain Late October/Early November Flood Events:


1. Event Details:
   ID: gdacs-event-1102983
   Title: Flood in Spain
   Date: 2024-10-27T15:00:00Z
   Collection: gdacs-events
   Countries: ['ESP']
   Hazard Codes: ['FL']
   Correlation ID: 20241027-ESP-NAT-HYD-FLO-FLO-1-GCDB

2. Event Details:
   ID: glide-event-FL-2024-000199-ESP
   Title: 
   Date: 2024-10-27T00:00:00Z
   Collection: glide-events
   Countries: ['ESP']
   Hazard Codes: ['nat-hyd-flo-flo', 'FL']
   Correlation ID: 20241027-ESP-NAT-HYD-FLO-FLO-1-GCDB

3. Event Details:
   ID: emdat-event-2024-0796-ESP
   Title: Flood (General) in Spain
   Date: 2024-10-27T00:00:00Z
   Collection: emdat-events
   Countries: ['ESP']
   Hazard Codes: ['nat-hyd-flo-flo']
   Correlation ID: 20241027-ESP-NAT-HYD-FLO-FLO-1-GCDB

4. Event Details:
   ID: gdacs-event-1102955
   Title: Flood in Spain
   Date: 2024-10-11T01:00:00Z
   Collection: gdacs-events
   Countries: ['ESP']
 

In [17]:
# Get correlation IDs from Spain flood events and find related hazards
if spain_flood_late_oct_events:
    # Collect unique correlation IDs
    correlation_ids = set()
    for event in spain_flood_late_oct_events:
        corr_id = event.properties.get('monty:corr_id')
        if corr_id:
            correlation_ids.add(corr_id)
    
    print(f"🔗 Found {len(correlation_ids)} unique correlation IDs from Spain flood events\n")
    print("="*80)
    
    # Search for hazards for each correlation ID
    all_hazards = []
    hazard_collections_used = ["glide-hazards", "gdacs-hazards", "usgs-hazards", "reference-hazards"]
    
    for idx, corr_id in enumerate(sorted(correlation_ids), 1):
        print(f"\n📍 CORRELATION ID {idx}/{len(correlation_ids)}: {corr_id}")
        print("="*80)
        
        # Search for related hazards using CQL2 filter
        hazard_filter = {
            "op": "=",
            "args": [
                {"property": "monty:corr_id"},
                corr_id
            ]
        }
        
        hazard_search = client.search(
            collections=hazard_collections_used,
            filter=hazard_filter,
            filter_lang="cql2-json",
            max_items=50
        )
        hazards = list(hazard_search.items())
        
        print(f"\n✅ Found {len(hazards)} hazard(s) across collections: {', '.join(hazard_collections_used)}\n")
        
        if hazards:
            all_hazards.extend(hazards)
            
            # Group hazards by collection for clearer display
            hazards_by_collection = {}
            for hazard in hazards:
                coll = hazard.collection_id
                if coll not in hazards_by_collection:
                    hazards_by_collection[coll] = []
                hazards_by_collection[coll].append(hazard)
            
            # Display hazards grouped by collection
            for collection_name, coll_hazards in sorted(hazards_by_collection.items()):
                print(f"📦 Collection: {collection_name} ({len(coll_hazards)} hazard{'s' if len(coll_hazards) > 1 else ''})")
                print("-" * 80)
                
                for i, hazard in enumerate(coll_hazards, 1):
                    print(f"\n  {i}. Hazard ID: {hazard.id}")
                    print(f"     Date: {hazard.properties.get('datetime', 'N/A')}")
                    print(f"     Title: {hazard.properties.get('title', 'N/A')}")
                    
                    # Display hazard detail if available
                    hazard_detail = hazard.properties.get('monty:hazard_detail', {})
                    if hazard_detail:
                        severity = hazard_detail.get('severity_value', 'N/A')
                        severity_unit = hazard_detail.get('severity_unit', '')
                        cluster = hazard_detail.get('cluster', 'N/A')
                        print(f"     Severity: {severity} {severity_unit}")
                        print(f"     Cluster: {cluster}")
                    else:
                        print(f"     Severity: N/A")
                        print(f"     Cluster: N/A")
                
                print()
        else:
            print("⚠️ No hazards found for this correlation ID")
    
    print("\n" + "="*80)
    print(f"\n🎯 FINAL SUMMARY")
    print("="*80)
    print(f"Total Correlation IDs: {len(correlation_ids)}")
    print(f"Total Hazards Found: {len(all_hazards)}")
    
    # Summary by collection
    if all_hazards:
        print(f"\n📊 Hazards by Collection:")
        collection_summary = {}
        for hazard in all_hazards:
            coll = hazard.collection_id
            collection_summary[coll] = collection_summary.get(coll, 0) + 1
        
        for coll, count in sorted(collection_summary.items()):
            print(f"  • {coll}: {count} hazard{'s' if count > 1 else ''}")
    
else:
    print("ℹ️ No Spain flood events found to search for hazards")

🔗 Found 2 unique correlation IDs from Spain flood events


📍 CORRELATION ID 1/2: 20241011-ESP-NAT-HYD-FLO-FLO-1-GCDB

✅ Found 1 hazard(s) across collections: glide-hazards, gdacs-hazards, usgs-hazards, reference-hazards

📦 Collection: gdacs-hazards (1 hazard)
--------------------------------------------------------------------------------

  1. Hazard ID: gdacs-hazard-1102955-1
     Date: 2024-10-11T01:00:00Z
     Title: Flood in Spain
     Severity: 0.5 GDACS Severity Score
     Cluster: nat-hyd-flo-flo


📍 CORRELATION ID 2/2: 20241027-ESP-NAT-HYD-FLO-FLO-1-GCDB

✅ Found 3 hazard(s) across collections: glide-hazards, gdacs-hazards, usgs-hazards, reference-hazards

📦 Collection: gdacs-hazards (2 hazards)
--------------------------------------------------------------------------------

  1. Hazard ID: gdacs-hazard-1102983-2
     Date: 2024-10-27T15:00:00Z
     Title: Flood in Spain
     Severity: 0.5 GDACS Severity Score
     Cluster: nat-hyd-flo-flo

  2. Hazard ID: gdacs-hazard-110298

### 5.6 Finding Impacts Associated with Spain Flood Events

Now let's search for impact data related to the Spain flood events:

In [18]:
# Search for impacts related to Spain flood events
if spain_flood_late_oct_events:
    print("🔍 Searching for impacts related to Spain flood events\n")
    print("="*80)
    
    # Define impact filter for Spain floods
    impact_filter = {
        "op": "and",
        "args": [
            {
                "op": "a_contains",
                "args": [
                    {"property": "monty:country_codes"},
                    "ESP"
                ]
            },
            {
                "op": "a_overlaps",
                "args": [
                    {"property": "monty:hazard_codes"},
                    ["nat-hyd-flo-flo", "FL"]
                ]
            },
            {
                "op": "t_intersects",
                "args": [
                    {"property": "datetime"},
                    {"interval": ["2024-10-20T00:00:00Z", "2024-11-15T23:59:59Z"]}
                ]
            }
        ]
    }
    
    # Search across impact collections
    impact_collections = ["gdacs-impacts", "emdat-impacts", "idmc-gidd-impacts", "idmc-idu-impacts"]
    
    impact_search = client.search(
        collections=impact_collections,
        filter=impact_filter,
        filter_lang="cql2-json",
        max_items=100
    )
    impacts = list(impact_search.items())
    
    print(f"✅ Found {len(impacts)} impact(s) across collections: {', '.join(impact_collections)}\n")

🔍 Searching for impacts related to Spain flood events

✅ Found 46 impact(s) across collections: gdacs-impacts, emdat-impacts, idmc-gidd-impacts, idmc-idu-impacts



In [19]:
# Diagnostic: Examine the actual structure of impact items
if impacts:
    print("🔍 DIAGNOSTIC: Examining Impact Data Structure\n")
    print("="*80)
    
    # Sample a few impacts from different collections
    sample_impacts = {}
    for impact in impacts[:10]:
        coll = impact.collection_id
        if coll not in sample_impacts:
            sample_impacts[coll] = impact
    
    for collection_name, impact in sample_impacts.items():
        print(f"\n📦 Collection: {collection_name}")
        print(f"Impact ID: {impact.id}")
        print("-" * 80)
        
        # Show all properties
        print("\n📋 All Properties:")
        for key, value in impact.properties.items():
            if isinstance(value, dict):
                print(f"  {key}:")
                for sub_key, sub_value in value.items():
                    print(f"    {sub_key}: {sub_value}")
            else:
                print(f"  {key}: {value}")
        
        # Show monty:impact_detail specifically
        impact_detail = impact.properties.get('monty:impact_detail', {})
        if impact_detail:
            print(f"\n⚡ monty:impact_detail structure:")
            for key, val in impact_detail.items():
                print(f"    {key}: {val}")
        else:
            print(f"\n⚠️ monty:impact_detail is empty or missing")
        
        print("\n" + "="*80)
else:
    print("No impacts to examine")

🔍 DIAGNOSTIC: Examining Impact Data Structure


📦 Collection: idmc-idu-impacts
Impact ID: idmc-idu-impact-28064168623-displaced
--------------------------------------------------------------------------------

📋 All Properties:
  roles: ['source', 'impact']
  title: Spain: Flood - Andalusia (Málaga) - 13/11/2024
  sources: Local Authorities
  datetime: 2024-11-13T00:00:00Z
  location: Alhaurín de la Torre, Malaga, Andalusia, Spain; Álora, Malaga, Andalusia, Spain; Cártama, Malaga, Andalusia, Spain; Málaga, Malaga, Andalusia, Spain; Pizarra, Malaga, Andalusia, Spain; Vélez-Málaga, La Axarquía, Malaga, Andalusia, Spain
  description:  **Spain: 4,210 displacements, 13 November - 14 November**    
 According to local authority, a total of 4210 people were displaced in Álora, Cártama, Alhaurín de la Torre, Pizarra, Málaga, Vélez-Málaga due to flood on 13 November.   
 [Local Authorities - 14 November 2024](https://www.juntadeandalucia.es/presidencia/portavoz/emergencias112/198838/suspension

In [20]:
# Create a DataFrame with all impact data
if impacts:
    impact_data = []
    
    for impact in impacts:
        impact_detail = impact.properties.get('monty:impact_detail', {})
        
        # Extract all relevant fields
        impact_record = {
            'Impact ID': impact.id,
            'Collection': impact.collection_id,
            'Correlation ID': impact.properties.get('monty:corr_id', 'N/A'),
            'Date': impact.properties.get('datetime', 'N/A'),
            'Title': impact.properties.get('title', 'N/A'),
            'Category': impact_detail.get('category', 'N/A') if impact_detail else 'N/A',
            'Type': impact_detail.get('type', 'N/A') if impact_detail else 'N/A',
            'Value': impact_detail.get('value', None) if impact_detail else None,
            'Unit': impact_detail.get('unit', 'N/A') if impact_detail else 'N/A',
            'Estimate Type': impact_detail.get('estimate_type', 'N/A') if impact_detail else 'N/A'
        }
        
        impact_data.append(impact_record)
    
    # Create DataFrame
    impacts_df = pd.DataFrame(impact_data)
    
    # Display summary
    print("Impact Data Summary")
    print("="*80)
    print(f"Total Impacts: {len(impacts_df)}")
    print(f"\nDataFrame Shape: {impacts_df.shape}")
    print(f"Columns: {list(impacts_df.columns)}\n")
    
    # Display the DataFrame
    print("="*80)
    print("\nAll Impacts Table:\n")
    display(impacts_df)
    
    # Summary statistics by category and type
    print("\n" + "="*80)
    print("\nSummary by Category and Type:\n")
    
    summary = impacts_df.groupby(['Category', 'Type']).agg({
        'Value': ['count', 'sum', 'mean', 'min', 'max']
    }).round(0)
    
    # Reset index to make Category and Type regular columns
    summary = summary.reset_index()
    
    # Flatten the multi-level column names
    summary.columns = ['Category', 'Type', 'Count', 'Sum', 'Mean', 'Min', 'Max']
    
    display(summary)
    
else:
    print("No impacts available to create DataFrame")

Impact Data Summary
Total Impacts: 46

DataFrame Shape: (46, 10)
Columns: ['Impact ID', 'Collection', 'Correlation ID', 'Date', 'Title', 'Category', 'Type', 'Value', 'Unit', 'Estimate Type']


All Impacts Table:



,Impact ID,Collection,Correlation ID,Date,Title,Category,Type,Value,Unit,Estimate Type
0,idmc-idu-impact-28064168623-displaced,idmc-idu-impacts,20241113-ESP-NAT-HYD-FLO-FLO-1-GCDB,2024-11-13T00:00:00Z,Spain: Flood - Andalusia (Málaga) - 13/11/2024,people,displaced_internal,4210.0,count,primary
1,idmc-gidd-impact-28064168623-Internal Displace...,idmc-gidd-impacts,20241113-ESP-MH0600-1-GCDB,2024-11-13T00:00:00Z,Internal Displacements-Person-Person-Spain: Fl...,people,displaced_internal,4210.0,count,primary
2,emdat-impact-2024-0839-ESP-total_dam,emdat-impacts,20241113-ESP-NAT-HYD-FLO-FLO-1-GCDB,2024-11-13T00:00:00Z,Flood (General) in Spain - total_dam,total_affected,cost,50000.0,count,primary
3,emdat-impact-2024-0839-ESP-total_affected,emdat-impacts,20241113-ESP-NAT-HYD-FLO-FLO-1-GCDB,2024-11-13T00:00:00Z,Flood (General) in Spain - total_affected,people,affected_total,500.0,count,primary
4,emdat-impact-2024-0839-ESP-no_affected,emdat-impacts,20241113-ESP-NAT-HYD-FLO-FLO-1-GCDB,2024-11-13T00:00:00Z,Flood (General) in Spain - no_affected,people,affected_total,500.0,count,primary
...,...,...,...,...,...,...,...,...,...,...
41,gdacs-hazard-1102983-1-A-missing-Spain-Valenci...,gdacs-impacts,20241027-ESP-NAT-HYD-FLO-FLO-1-GCDB,2024-10-27T15:00:00Z,Flood in Spain,people,missing,114.0,sendai,primary
42,emdat-impact-2024-0796-ESP-total_deaths,emdat-impacts,20241027-ESP-NAT-HYD-FLO-FLO-1-GCDB,2024-10-27T00:00:00Z,Flood (General) in Spain - total_deaths,people,death,232.0,count,primary
43,emdat-impact-2024-0796-ESP-total_dam,emdat-impacts,20241027-ESP-NAT-HYD-FLO-FLO-1-GCDB,2024-10-27T00:00:00Z,Flood (General) in Spain - total_dam,total_affected,cost,11000000.0,count,primary
44,emdat-impact-2024-0796-ESP-total_affected,emdat-impacts,20241027-ESP-NAT-HYD-FLO-FLO-1-GCDB,2024-10-27T00:00:00Z,Flood (General) in Spain - total_affected,people,affected_total,36115.0,count,primary




Summary by Category and Type:



,Category,Type,Count,Sum,Mean,Min,Max
0,buildings,damaged,2,9000.0,4500.0,4500.0,4500.0
1,buildings,destroyed,1,1.0,1.0,1.0,1.0
2,people,affected_total,4,73230.0,18308.0,500.0,36115.0
3,people,assisted,1,26.0,26.0,26.0,26.0
4,people,death,10,738.0,74.0,1.0,232.0
5,people,displaced_internal,15,12285.0,819.0,0.0,4210.0
6,people,missing,3,233.0,78.0,5.0,114.0
7,people,relocated,4,1034.0,258.0,40.0,447.0
8,people,shelter_emergency,4,4293.0,1073.0,8.0,3701.0
9,total_affected,cost,2,11050000.0,5525000.0,50000.0,11000000.0


---

## 9. Summary and Key Takeaways

### What We've Accomplished:

In this notebook, we focused on **querying and retrieving disaster data** from the Montandon STAC API, with a detailed case study on Spain flood events (October-November 2024).

#### 1. **API Connection & Collections**
   - Connected to the Montandon STAC API staging environment
   - Explored 30+ collections organized by data source and type (events, hazards, impacts)
   - Understood the structure: GDACS, EMDAT, GLIDE, IDMC, USGS data sources

#### 2. **Basic Query Techniques**
   - Simple collection browsing with `fetch_collection_items()`
   - Direct API requests using HTTP requests
   - Parameter-based filtering with `limit` and collection selection

#### 3. **Advanced CQL2 Filtering**
   - **Temporal filtering**: Date range queries (e.g., October 20 - November 15, 2024)
   - **Spatial filtering**: Bounding box queries (e.g., Europe region)
   - **Property filtering**: Country codes (`monty:country_codes`), hazard codes (`monty:hazard_codes`)
   - **Complex operators**: `a_contains`, `a_overlaps`, `t_intersects`

#### 4. **Event-Hazard-Impact Correlation**
   - Found 4 flood events in Spain using CQL2 filters
   - Retrieved 4 related hazards using correlation IDs (`monty:corr_id`)
   - Discovered 46 impact records across 4 collections:
     - 7 from EMDAT impacts
     - 20 from GDACS impacts
     - 14 from IDMC-GIDD impacts
     - 5 from IDMC-IDU impacts

#### 5. **Data Structure Understanding**
   - Examined `monty:impact_detail` structure with fields: category, type, value, unit, estimate_type
   - Identified impact types: displaced_internal, cost, etc.
   - Mapped impact categories: people, total_affected, buildings, etc.

#### 6. **Data Organization**
   - Converted 46 impact items into a pandas DataFrame
   - Organized data with 10 columns: Impact ID, Collection, Correlation ID, Date, Title, Category, Type, Value, Unit, Estimate Type
   - Created a summary table grouping impacts by Category and Type with counts, sums, means, min/max values

### Key Properties Reference:

| Property | Description | Example |
|----------|-------------|---------|
| `monty:corr_id` | Correlation ID linking events, hazards, impacts | `20241113-ESP-NAT-HYD-FLO-FLO-1-GCDB` |
| `monty:country_codes` | ISO 3166-1 alpha-3 country codes | `['ESP']` for Spain |
| `monty:hazard_codes` | Hazard classification codes | `['nat-hyd-flo-flo', 'FL']` for floods |
| `monty:impact_detail` | Detailed impact metrics (category, type, value, unit, estimate_type) | See impact data |
| `datetime` | Event/hazard/impact timestamp | ISO 8601 format |

### Case Study Results: Spain Floods October-November 2024

**Events Found**: 4
- GDACS Event 1102983, GLIDE Event FL-2024-000199-ESP, EMDAT Event 2024-0796, GDACS Event 1102955

**Hazards Found**: 4 total
- 3 from GDACS hazards
- 1 from GLIDE hazards

**Impacts Found**: 46 total
- Primary impact types: Internal displacements (4,210+ people), economic costs (50,000+), affected population counts
- Data spans November 13-14, 2024 with focus on Andalusia region (Málaga, Granada areas)



---

## Practice Exercise

Try this on your own:

1. **Search for flood events** in your country or region of interest
2. **Filter by date range** (e.g., last 2 years)
3. **Convert to DataFrame** and analyze the patterns
4. **Find related hazards** using correlation IDs
5. **Export the results** for your records

Use the code cells below to practice:

In [21]:
# Your practice code here
# Example: Search for floods in your region


---

**End of Notebook 1**

Continue to the next notebook for advanced visualization and spatial analysis!